# 8.3.1 Workshop on Topic Modelling (Part 1, Prereqs)

```
date: "Block 08"
author: "Daniel Lawson"
email: dan.lawson@bristol.ac.uk
output: html_document
version: 2.0.0
```

## NLP Environment

Here we set up the NLP environment in python.


## The libraries that are needed

The main library for this is called "gensim". However, there are several libraries that implement better natural language processing technologies.  The main facility is something called "stemming" or "lemmatizing". 

https://blog.bitext.com/what-is-the-difference-between-stemming-and-lemmatization/

* **Stemming**: Cutting off the start and end of words to make (hopefully) consistent word stems
* **Lemmatization**: Looking up the correct word-roots in dictionaries to find the morphological root of words

Stemming may work across languages and dialects, and is computationally cheaper. However, it may merge words that are functionally different; for example, *operation*, *operational*, *operand* and *opera* might all become *oper*!

Lemmatization works only as well as its dictionary and requires large databases to function. It would be preferred for many language tasks.

An important additional step is to remove **stop words**. These are common words that link other words but have little intrinsic meaning by themselves, such as "the", "it", "on", "and", etc.

## Installation of software

Here we install the required packages, nltk and gensim. Note that this is not completing the setup for these as they contain sub-modules that need configuring.

In [11]:
!pip install nltk
!pip install gensim
!pip install pyLDAvis

Now we'll check our gensim version.

You can ignore this; but, you need gensim v3, not v0, for some of the functions to work. 

The command-line update is "conda install -c anaconda gensim"

Now we are ready to test that we can load all software we need:

In [2]:
import pickle
import pandas as pd
import requests
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

import nltk

In [3]:
print (gensim.__version__)
## If you need to reinstall and reload:
##from importlib import reload
##reload(gensim)

4.2.0


Finally, we need to download the nltk modules that are needed. 

In [4]:
### Do this once! Then leave commented next time you run the script.
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('popular') # Get the "popular" package with the advanced WordNet dictionary in it. There are ways to avoid this is you want.

[nltk_data] Downloading package stopwords to /Users/madjl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/madjl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/madjl/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/madjl/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/madjl/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/madjl/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    | 

True

## Data

First, load the data. This idea comes from [Susan Li on Towards Data Science](https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24) and the data is direct from [Kaggle million headlines](https://www.kaggle.com/therohk/million-headlines/data). Because the documents are short, this is a useful dataset for teaching.

We will download it from the [Data Science GitHub Data Directory](https://github.com/dsbristol/dst/tree/master/data) using the [Direct Download link](https://github.com/dsbristol/dst/blob/master/data/abcnews-date-text.csv.gz?raw=true)
using the python below.

We will also get the [dst-block7-lda.zip](https://github.com/dsbristol/dst/blob/master/data/dst-block7-lda.zip?raw=true) zip file which contains some intermediate results that we don't want to have to run in real time. They take about 10-15 mins to regenerate in total and you will have the code to generate them.

In [5]:
import os
url = 'https://github.com/dsbristol/dst/blob/master/data/abcnews-date-text.csv.gz?raw=true'
r = requests.get(url, allow_redirects=True)
open(os.path.join('..', 'data', 'abcnews-date-text.csv.gz'), 'wb').write(r.content)

19296792

In [6]:
data = pd.read_csv(os.path.join('..', 'data', 'abcnews-date-text.csv.gz'), 
                   compression='gzip',error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text[0:100000]

/var/folders/sd/6b6ndf8s69l3glyl3t0xl7zr0000gq/T/ipykernel_47717/3200708286.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(os.path.join('..', 'data', 'abcnews-date-text.csv.gz'),


Summaries of the data:

In [7]:
print(len(documents))
print(documents[:5])

100000
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [8]:
url = 'https://github.com/dsbristol/dst/blob/master/data/dst-block7-lda.zip?raw=true'
r = requests.get(url, allow_redirects=True)
open(os.path.join('..', 'data', 'dst-block7-lda.zip'), 'wb').write(r.content)

6239995

Check that the file is not corrupted: it should have a hash of 793d48054fa8ec271ed6c683295f1122

This is important because the zip files contains pkl files which are vulnerable to malicious use.

In [9]:
import hashlib
print(hashlib.md5(r.content).hexdigest()) 

793d48054fa8ec271ed6c683295f1122


Now we'll unzip these files so you don't have to regenerate them:

In [10]:
import zipfile
with zipfile.ZipFile(os.path.join('..', 'data', 'dst-block7-lda.zip'), 'r') as zip_ref:
    zip_ref.extractall(os.path.join('..', 'data'))

That is it, we're ready to do NLP in anger.